# **Summarisation & Evasion Notebook (LB)**

# **1. Objective**

- Summarise banker answers into short, PRA relevant insights.
- Generate an evasion score to tag summaries.
- RAG pipeline to bring in relevant external documents (e.g. PRA risk definitions, regulatory news).
- Optional extension: validate flagged risks with external/regulatory news.

# **2. Set up Workspace**

In [128]:
# Import libraries
# Core python
import os
import numpy as np
import pandas as pd
import re
import json
from pathlib import Path
from typing import List, Dict, Any 
import csv

# NLP & Summarisation
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
import spacy
from llama_cpp import Llama 

# Evaluation
from rouge_score import rouge_scorer
import evaluate
from bert_score import score as bertscore 

# Retrieval
from sentence_transformers import SentenceTransformer 
import faiss
import chromadb
import langchain
import llama_index

# ML
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Visualisations
import matplotlib.pyplot as plt
import seaborn as sns 

# **3. Load the dataset**

In [129]:
# Load the dataset.
jpm_2025_df = pd.read_csv('../data/processed/jpm/all_jpm_2025.csv')

# View the data.
jpm_2025_df.head()

,question_number,answer_number,speaker_name,role,company,content,year,quarter,source_pdf
0,1,NaN,Ken Usdin,analyst,Autonomous Research,"Good morning, Jeremy. Wondering if you could s...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
1,1,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Sure, Ken. So I mean, at a high level, I would...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
2,2,NaN,Ken Usdin,analyst,Autonomous Research,Yeah. And just one question on the NII ex. Mar...,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
3,2,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Yeah, that's a good question, Ken. You're righ...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...
4,2,2.0,Jamie Dimon,Chairman & Chief Executive Officer,JPMorganChase,In the curve basically.,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...


# **4. Preprocessing**

In [130]:
# View speaker roles.
jpm_2025_df['role'].unique()

array(['analyst', 'Chief Financial Officer',
       'Chairman & Chief Executive Officer',
       'And then some. Theres a lot of value added.', 'Okay'],
      dtype=object)

In [131]:
# View rows with invalid roles.
valid_roles = 'analyst', 'Chief Financial Officer', 'Chairman & Chief Executive Officer'
invalid_roles_df = jpm_2025_df[~jpm_2025_df['role'].isin(valid_roles)]

# Number of rows with invalid roles.
print('Number of rows:', invalid_roles_df.shape[0])

# View the rows.
invalid_roles_df.head()

Number of rows: 2


,question_number,answer_number,speaker_name,role,company,content,year,quarter,source_pdf
201,35,5.0,"Chief Financial Officer, JPMorganChase",And then some. Theres a lot of value added.,JPMorganChase,"Yeah. And obviously, I mean, we're not going t...",2025,Q2,data/raw/jpm/jpm-2q25-earnings-call-transcript...
205,36,3.0,"Chief Financial Officer, JPMorganChase",Okay,there you have it.,"But it's not like I thought it would do badly,...",2025,Q2,data/raw/jpm/jpm-2q25-earnings-call-transcript...


In [132]:
# Input the correct role information.
jpm_2025_df.at[205, 'role'] = 'Chief Financial Officer'
jpm_2025_df.at[209, 'role'] = 'Chief Financial Officer'

# Verify the roles have been updated.
jpm_2025_df['role'].unique()

array(['analyst', 'Chief Financial Officer',
       'Chairman & Chief Executive Officer',
       'And then some. Theres a lot of value added.'], dtype=object)

In [133]:
# Define role mapping.
role_map = {
    'analyst': 'analyst',
    'Chief Financial Officer': 'banker',
    'Chairman & Chief Executive Officer': 'banker'
}

# Apply to dataset.
jpm_2025_df['role_normalised'] = jpm_2025_df['role'].map(role_map)

In [134]:
# View the dataset.
jpm_2025_df.head()

,question_number,answer_number,speaker_name,role,company,content,year,quarter,source_pdf,role_normalised
0,1,NaN,Ken Usdin,analyst,Autonomous Research,"Good morning, Jeremy. Wondering if you could s...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,analyst
1,1,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Sure, Ken. So I mean, at a high level, I would...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,banker
2,2,NaN,Ken Usdin,analyst,Autonomous Research,Yeah. And just one question on the NII ex. Mar...,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,analyst
3,2,1.0,Jeremy Barnum,Chief Financial Officer,JPMorganChase,"Yeah, that's a good question, Ken. You're righ...",2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,banker
4,2,2.0,Jamie Dimon,Chairman & Chief Executive Officer,JPMorganChase,In the curve basically.,2025,Q1,data/raw/jpm/jpm-1q25-earnings-call-transcript...,banker


# **5. Summarisation**

## **5.1 Baseline**
- Model = BART (not instruction tuned)

In [135]:
# Filter data to banker answers only.
banker_answers = jpm_2025_df[jpm_2025_df['role_normalised'] == 'banker']['content'].tolist()
print(banker_answers[0][:200])

Sure, Ken. So I mean, at a high level, I would say that obviously, some of the salient news flow is quite recent. So, we've done some soundings and some checking both on the consumer side and on the w


In [136]:
# Summarisation baseline (BART)
summariser = pipeline('summarization', model='facebook/bart-large-cnn')

sample_text = banker_answers[0]
summary = summariser(sample_text, max_length=80, min_length=30, do_sample=False)
print('Original:', sample_text[:400])
print('Summary:', summary[0]['summary_text'])

Device set to use mps:0


Original: Sure, Ken. So I mean, at a high level, I would say that obviously, some of the salient news flow is quite recent. So, we've done some soundings and some checking both on the consumer side and on the wholesale side. I think on the consumer side, the thing to check is the spending data. And to be honest, the main thing that we see there, what would appear to be a certain amount of frontloading of sp
Summary: The main thing that we see there, what would appear to be a certain amount of frontloading of spending ahead of people expecting price increases from tariffs. So ironically, that's actually somewhat supportive, all else equal. In terms of our corporate clients, obviously, they've been reacting to the changes in tariff policy.


In [137]:
# Prompt conditioning to make PRA relevant.
prompt = "Summarise this answer, focusing on risk, capital and evasion of detail: " + sample_text
summary = summariser(prompt, max_length=80, min_length=30)
print('Original:', sample_text[:400])
print('Summary:', summary[0]['summary_text'])

Original: Sure, Ken. So I mean, at a high level, I would say that obviously, some of the salient news flow is quite recent. So, we've done some soundings and some checking both on the consumer side and on the wholesale side. I think on the consumer side, the thing to check is the spending data. And to be honest, the main thing that we see there, what would appear to be a certain amount of frontloading of sp
Summary: Corporates are taking a wait-and-see approach to tariff policy. Some sectors are going to be much more exposed than others. Small business and smaller corporates are probably a little more challenged.


## **5.2 RAG**

### **Action point:** test different model to try prompting with PRA context**

Retrieve PRA risk categories to give greater PRA focus to summaries (local RAG loop).
- measure cosine similarity between transcript chunks and PRA risk categories (vectors)
- retrieve the top 2-3 most relevant risk categories 
- prepend them to the summarisation prompt to make summaries PRA-aligned instead of just summarised answers

- Attempting to use BART resulted in prompt echoing.
- New attempt using Mistral-7B-Instruct.
- Using sentence-BERT vs TF-IDF for vectorisation.

In [138]:
# Function to remove whitespace in text.
def clean_text(text: str):
    return re.sub(r'\s+', ' ', text).strip()

In [139]:
# Function to split the transcript into smaller chunks.
def chunk_text(text: str, max_chars: int = 6000):
    sentences = re.split(r'(?<=[.!?])\s+', text.strip()) # split into sentences 
    chunks, current_chunk, current_len = [], [], 0 # list of chunks, sentences collecting for current chunk, character count for current chunk

    for s in sentences:
        if current_len + len(s) + 1 <= max_chars: # if the characters of current chunk + new sentence is below the limit:
            current_chunk.append(s) # add sentence to current chunk 
            current_len += len(s) + 1 # update running character count 
        
        else: # if the characters is above the limit:
            chunks.append(' '.join(current_chunk)) # add the current chunk to the final chunk list
            current_chunk, current_len = [s], len(s) # start a new chunk containing the sentence and update current len

    if current_chunk:
        chunks.append(' '.join(current_chunk)) # add any sentences in current chunk after loop ends 

    return chunks 

In [140]:
# # Load PRA risk categories.
# pra_risk_df = pd.read_csv('../data/RAG-resources/PRA_risk_categories.csv')
# pra_risk_df['doc'] = pra_risk_df['category'] + ': ' + pra_risk_df['definition']

# # View dataframe.
# pra_risk_df

In [141]:
def load_pra_categories(path: Path):
    with open(path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        return [
            (row.get('category', '').strip(), [row.get('definition', '').strip()])
            for row in reader if row.get('category')
        ]

In [142]:
# Build a Sentence-BERT embedding index for PRA categories.
def build_embedding_index(pra_categories):
    embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    pra_risk_embeddings = embedder.encode(pra_risk_df['doc'], batch_size=32, normalize_embeddings=True)

    return embedder, np.asarray(pra_risk_embeddings)

In [143]:
# Function to find the relevant PRA categories to the transcript chunks.
def find_rel_categories(chunk, pra_categories, embedder, pra_risk_embeddings, top_k=2):
    query_vec = embedder.encode([chunk], normalize_embeddings=True) # turns chunk into embedding
    sims = cosine_similarity(query_vec, pra_risk_embeddings).ravel() # compares the chunk to each category doc 
    top_indices = np.argsort(-sims)[:top_k] # sorts scores descending and selected top k cateogories 

    return [pra_categories[i] for i in top_indices]

In [144]:
def extract_json_block(model_output: str) -> str:
    """If the model adds extra text, grab the first JSON object."""
    a = model_output.find("{")
    b = model_output.rfind("}")
    return model_output[a:b+1] if a != -1 and b != -1 and b > a else model_output

In [145]:
# Function to summarise the chunk.
def summarise_chunk(model, chunk, relevant_categories, max_evidence=5):
    lines = []
    for name, definition in relevant_categories:
        lines.append(f'- {name}:')
        for d in definition:
            lines.append(f'- {d}')
    notes_block = '\n'.join(lines)

    system_prompt = (
        'You are a Data Scientist producing PRA-aligned summaries.'
        'Use only the transcript text as evidence. Map content to the PRA categories provided.'
    )

    user_prompt = f"""
TRANSCRIPT:
{chunk}

PRA NOTES:
{notes_block}

TASK:
Return STRICT JSON with keys:
- evidence: list of up to {max_evidence} bullet strings
- pra categories: list of objects {{ 'name': '...', 'why': '...' }}
- summary: 4-6 sentences, neutral tone

RULES:
- Do not invent facts. If evidence is lacking, use a single bullet 'Insufficient evidence'.
- Only choose categories supported by the evidence.
- No markdown. Output JSON only.
"""
    
    response = model.create_chat_completion(
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': clean_text(user_prompt)}
        ],
        temperature=0.2,
        max_tokens=512,
        repeat_penalty=1.1
    ) 

    text = response['choices'][0]['message']['content'].strip()
    try:
        return json.loads(exytact_json_block(text))
    except Exception:
        return {'raw': text}

In [146]:
# Define variables.
MODEL_PATH = '/Users/laurenbrixey/Documents/Data Science Career Accelerator/Project Submissions/Course 3/topic_project_4.1/mistral-7b-instruct-v0.1.Q4_K_M.gguf'
PRA_NOTES_PATH = '../data/RAG-resources/PRA_risk_categories.csv'
TRANSCRIPT_PATH = '../data/processed/jpm/all_jpm_2025.csv'
OUTPUT_PATH = "jpm_mistral_pra_summary.json"
TOP_K = 5

In [ ]:
# Runner.
pra_categories = load_pra_categories(Path(PRA_NOTES_PATH)) # load pra category file
embedder, category_embeddings = build_embedding_index(pra_categories) # embed pra categories

transcript_text = Path(TRANSCRIPT_PATH).read_text() # load the transcript text
transcript_chunks = chunk_text(transcript_text) # split the transcript into smaller chunks

model = Llama(model_path=str(Path(MODEL_PATH).expanduser()), n_ctx=8192, n_gpu_layers=1, chat_format='mistral-instruct')

results = []
for i, chunk in enumerate(transcript_chunks, 1):
    top_categories = find_rel_categories(chunk, pra_categories, embedder, category_embeddings)
    summary_result = summarise_chunk(model, chunk, top_categories)
    results.append({'chunk': i, 'result': summary_result})

combined_summary = ' '.join(r['result'].get('summary', '') for r in results)
final_output = {'chunks': results, 'combined_summary': combined_summary.strip()}

Path(OUTPUT_PATH).write_text(json.dumps(final_output, indent=2, ensure_ascii=False))
print(json.dumps(final_output, indent=2, ensure_ascii=False))

llama_model_load_from_file_impl: using device Metal (Apple M3) - 935 MiB free
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/laurenbrixey/Documents/Data Science Career Accelerator/Project Submissions/Course 3/topic_project_4.1/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forwa

In [ ]:
# # Load PRA categories and descriptions from csv file.
# def load_pra_categories(path: Path):
#     with open(path, newline='', ecoding='utf-8') as f:
#         reader = csv.DictReader(f)
        
#         return [
#             (row.get('category', '').strip(), [row.get('definition', '').strip()])
#             for row in reader if row.get('category')
#         ]

In [ ]:
# # Create TF-IDF index for PRA categories.
# def build_tfidf_index(pra_categories):
#     docs = [f'{name} {' '.join(definition)}'.strip() for name, definition in pra_categories] # create a single document 
#     vectorizer = TfidfVectorizer(stop_words='english') # create a tf-idf vectoriser, removes stopwords 
#     tfidf_matrix = vectorizer.fit_transform(docs) # creates tf-idf matrices from docs 
#     return vectorizer, tfidf_matrix

In [ ]:
# # Create embeddings of each doc (defintion + risk).
# embed = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# pra_risk_embeddings = embed.encode(pra_risk_df['doc'].tolist(), normalize_embeddings=True).astype('float32')
# index = faiss.IndexFlatIP(pra_risk_embeddings.shape[1])
# index.add(pra_risk_embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# # Retrieve the most relevant risk categories + defintions 
# def retrieve_pra_context(query_text: str, k: int = 3) -> pd.DataFrame:
#     q = embed.encode([query_text], normalize_embeddings=True).astype('float32')
#     scores, idxs = index.search(q, k)
#     out = pra_risk_df.iloc[idxs[0]].copy()
#     out['score'] = scores[0]
#     return out 

In [ ]:
# # Convert retrieved risk doc into a context block to prompt the model.
# def make_context_block(retrieved_df: pd.DataFrame) -> str:
#     lines = [f"- {r['category']}: {r['definition']}" for _, r in retrieved_df.iterrows()]
#     return '\n'.join(lines)

In [ ]:
# # Convert retrieved risk doc into a pra keyword to prompt the model.
# def pra_keywords(retrieved_df: pd.DataFrame, top_n: int = 3) -> str:
#     return ', '.join(retrieved_df['category'].head(top_n).tolist())

In [ ]:
# # Chunking the banker's answers to avoid truncation.
# def split_into_chunks(text: str, max_chunk_size: int = 1800):

#     # split text into sentences
#     sentences = re.split(r'(?<=[.!?])\s+', text.strip())

#     # list to store chunks
#     chunks = []
#     current_chunk = ''
#     for s in sentences:
#         if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:
#             current_chunk = f'{current_chunk} {sentence}'.strip()
#         else:
#             if current_chunk:
#                 chunks.append(current_chunk)
#             current_chunk = sentence 
    
#     if current_chunk: 
#         chunks.append(current_chunk)

#     return chunks

In [ ]:
# # Function to create a PRA risk category focused summary.
# bart = pipeline('summarization', model = 'facebook/bart-large-cnn')

# def bart_pra_focused_summary(answer_text: str, retrieved_df: pd.DataFrame, min_len: int = 40) -> str:
#     keywords = pra_keywords(retrieved_df, top_n=3)

#     # input: banker answer + PRA keyword hint 
#     input_text = f"{answer_text}\n\n[PRA focus: {keywords}]"

#     # number of input tokens
#     input_ids = bart.tokenizer.encode(input_text, truncation=True)
#     input_len = len(input_ids)

#     max_new = min(120, max(20, int(input_len * 0.7)))
#     min_new = min(60, int(input_len * 0.3))

#     # generate summaries
#     output = bart(
#         input_text,
#         max_new_tokens=max_new,
#         min_length=min_new,
#         do_sample=False,
#         num_beams=4,
#         no_repeat_ngram_size=3,
#         truncation=True
#     )
    
#     return output[0]['summary_text']
    
    
    
#     # top = retrieve_pra_context(answer_text, k=k)
#     # context = make_context_block(top)

#     # prompt = (
#     #     'Summarise the following banker answer for PRA stakeholders.\n'
#     #     'Focus on concrete details related to the most relevant risk categories below.\n\n'
#     #     'Relevant PRA risk categories:\n'
#     #     f'{context}\n\n'
#     #     'Banker answer:\n'
#     #     f'{answer_text}\n\n'
#     #     'Return 2 bullet points with a summary and then how it related to PRA risks'
#     # )

#     # out = summariser(prompt, max_length=max_len, min_length=min_len, do_sample=False)

#     # return out[0]['summary_text']

Device set to use mps:0


In [ ]:
# # Wrapper function.
# def pra_focused_summary(answer_text: str, k: int = 3) -> str:
#     top = retrieve_pra_context(answer_text, k=k)
#     return bart_pra_focused_summary(answer_text, top)

In [ ]:
# # Test on a sample.
# banker_answers_df = jpm_2025_df[jpm_2025_df['role_normalised'] == 'banker'][['content']]
# banker_answers_df_sample = banker_answers_df.head(25).copy()
# banker_answers_df_sample['pra_summary'] = banker_answers_df_sample['content'].apply(pra_focused_summary)

In [ ]:
# display(banker_answers_df_sample['pra_summary'])

# **6. Evasion Scoring**